In [ ]:
import pandas as pd
pd.test()

Running unit tests for pandas
pandas version 0.18.1
numpy version 1.11.1
pandas is installed in //anaconda/lib/python3.5/site-packages/pandas
Python version 3.5.2 |Anaconda 4.2.0 (x86_64)| (default, Jul  2 2016, 17:52:12) [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
nose version 1.3.7


//anaconda/lib/python3.5/imp.py:216: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
  return _load(spec)
//anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
..................................................................................................S.......S.......S.......S........................................................................................................................................................................................................................................................................................................................